In [1]:
import os
import pandas as pd
import tensorflow as tf

from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import concatenate
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten

from tensorflow.keras.activations import relu, softmax
from tensorflow.keras.preprocessing.image import ImageDataGenerator

### Config

In [2]:
train_meta_data = '../train.csv'
train_data_dir = '../input/paddy-disease-classification/train_images'
epochs = 100
lr = 1e-3
valid_split = 0.2
input_size = 224
batch_size = 32
classes = 10
initializer = tf.keras.initializers.HeUniform()
optimizer = tf.keras.optimizers.Adam(learning_rate=lr)
loss = tf.keras.losses.categorical_crossentropy

In [ ]:
early_stop = tf.keras.callbacks.EarlyStopping(patience=10,
                                              monitor='val_loss',
                                              restore_best_weights=True)

### Inception v1.0

In [ ]:
def inception(x, filters, projection, init=initializer, name=None):
    f_1x1, f_3x3, f_3x3_reduce, f_5x5, f_5x5_reduce = filters
    x1 = Conv2D(filters=f_1x1,
                kernel_size=(1, 1),
                kernel_initializer=init,
                strides=(1, 1),
                activation=relu,
                padding='same')(x)
    x3_reducer = Conv2D(filters=f_3x3_reduce,
                        kernel_size=(1, 1),
                        kernel_initializer=init,
                        strides=(1, 1),
                        activation=relu,
                        padding='same')(x)
    x5_reducer = Conv2D(filters=f_5x5_reduce,
                        kernel_size=(1, 1),
                        kernel_initializer=init,
                        strides=(1, 1),
                        activation=relu,
                        padding='same')(x)
    pool = MaxPooling2D(pool_size=(3, 3), strides=(1, 1), padding='same')(x)

    x3 = Conv2D(filters=f_3x3,
                kernel_size=(3, 3),
                kernel_initializer=init,
                strides=(1, 1),
                activation=relu,
                padding='same')(x3_reducer)
    x5 = Conv2D(filters=f_5x5,
                kernel_size=(5, 5),
                kernel_initializer=init,
                strides=(1, 1),
                activation=relu,
                padding='same')(x5_reducer)
    proj = Conv2D(filters=projection,
                  kernel_size=(1, 1),
                  kernel_initializer=init,
                  strides=(1, 1),
                  activation=relu,
                  padding='same')(pool)

    x = concatenate([x1, x3, x5, proj], axis=3, name=name)

    return x


def model_builder(shape, classes):
    input_layer = Input(shape=shape)
    x = Conv2D(filters=64, kernel_size=(7, 7), kernel_initializer=initializer, strides=(2, 2), activation=relu,
               padding='same')(input_layer)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = BatchNormalization()(x)
    x = Conv2D(filters=64, kernel_size=(1, 1), strides=(1, 1), activation=relu, padding='same')(x)
    x = Conv2D(filters=192, kernel_size=(3, 3), strides=(1, 1), activation=relu, padding='same')(x)
    x = BatchNormalization()(x)
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = inception(x, [64, 128, 96, 32, 16], projection=32, name='inception_3a')
    x = inception(x, [128, 192, 128, 96, 32], projection=64, name='inception_3b')
    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = inception(x, [192, 208, 96, 48, 16], projection=64, name='inception_4a')

    # auxiliary path 1
    aux_1 = AveragePooling2D(pool_size=(5, 5), strides=(3, 3), padding='valid')(x)
    aux_1 = Conv2D(filters=128, kernel_size=(1, 1), kernel_initializer=initializer, strides=(1, 1), activation=relu,
                   padding='valid')(aux_1)
    aux_1 = Dense(units=1024, activation=relu)(aux_1)
    aux_1 = Dropout(rate=0.7)(aux_1)
    aux_1 = Flatten()(aux_1)
    aux_out1 = Dense(units=classes, activation=softmax, name='aux_out1')(aux_1)

    x = inception(x, [160, 224, 112, 64, 24], projection=64, name='inception_4b')
    x = inception(x, [128, 256, 128, 64, 24], projection=64, name='inception_4c')
    x = inception(x, [112, 288, 144, 64, 32], projection=64, name='inception_4d')
    x = inception(x, [256, 320, 160, 128, 32], projection=128, name='inception_4e')

    # auxiliary path 2
    aux_2 = AveragePooling2D(pool_size=(5, 5), strides=(3, 3), padding='valid')(x)
    aux_2 = Conv2D(filters=128, kernel_size=(1, 1), kernel_initializer=initializer, strides=(1, 1), activation=relu,
                   padding='valid')(aux_2)
    aux_2 = Dense(units=1024, activation=relu)(aux_2)
    aux_2 = Dropout(rate=0.7)(aux_2)
    aux_2 = Flatten()(aux_2)
    aux_out2 = Dense(units=classes, activation=softmax, name='aux_out2')(aux_2)

    x = MaxPooling2D(pool_size=(3, 3), strides=(2, 2), padding='same')(x)
    x = inception(x, [256, 320, 160, 128, 32], projection=128, name='inception_5a')
    x = inception(x, [384, 384, 192, 128, 48], projection=128, name='inception_5b')
    x = AveragePooling2D(pool_size=(7, 7), strides=(1, 1))(x)
    x = Dropout(rate=0.4)(x)
    x = GlobalAveragePooling2D()(x)
    output_layer = Dense(units=classes, activation=softmax, name='main_out')(x)

    model = Model(input_layer, [output_layer, aux_out1, aux_out2])
    model.compile(optimizer=optimizer,
                  loss=loss,
                  loss_weights={'main_out': 1, 'aux_out1': 0.3, 'aux_out2': 0.3},
                  metrics=['accuracy'])

    return model

### Config data loaders

In [ ]:
generator = ImageDataGenerator(rescale=1 / 255,
                               validation_split=valid_split)

train_data = generator.flow_from_directory(directory=train_data_dir,
                                           target_size=(input_size, input_size),
                                           batch_size=batch_size)

### Model

In [ ]:
model = model_builder(shape=(input_size, input_size, 3), classes=classes)

In [ ]:
model.summary()

In [ ]:
tf.keras.utils.plot_model('baseline_inception.png')

In [ ]:
history = model.fit(train_data,
                    epochs=epochs,
                    callbacks=[early_stop])